<a href="https://colab.research.google.com/github/KadinRelefourd/neuralnetworkclassifier/blob/main/classify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [34]:
import os
import sys
import cv2  # OpenCV for image processing
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader

Prepare Training and Testing data

In [35]:
transform = transforms.ToTensor()

train_data = torchvision.datasets.CIFAR10(
    root='./data.cifar10',
    train=True,
    transform=transform,
    download=True
)
train_loader = DataLoader(dataset=train_data, batch_size=64, shuffle=True)

test_data = torchvision.datasets.CIFAR10(
    root='./data.cifar10',
    train=False,
    transform=transform,
    download=True
)
test_loader = DataLoader(dataset=test_data, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


Build model

In [36]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # CIFAR-10 images are 32x32x3
        self.fc1 = nn.Linear(32 * 32 * 3, 512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, 64)
        self.bn4 = nn.BatchNorm1d(64)
        # Output size is 10 because of the 10 classes in CIFAR-10
        self.fc5 = nn.Linear(64, 10)

    def forward(self, x):
        # flatten the tensor becuase the linear layer only accepts a vector.
        x = x.view(x.size(0), -1)
        #used relu as the activation function
        x = F.relu(self.bn1(self.fc1(x)))
        #I applied batc norm for better training time and applied dropout
        # but had to lower the probability of dropout to increase accuracy
        # maybe because I have 3 layers instead of 2
        x = F.dropout(x, p=0.3)
        x = F.relu(self.bn2(self.fc2(x)))
        x = F.dropout(x, p=0.3)
        x = F.relu(self.bn3(self.fc3(x)))
        x = F.dropout(x, p = 0.3)
        x = F.relu(self.bn4(self.fc4(x)))
        x = self.fc5(x)
        return x

model = Net()
#crossEntropyLoss includes softmax and is the loss function you need for
#classification problems
loss_func = nn.CrossEntropyLoss()
#used SGD because adam gave me low accuracy and played with the hyperparametss
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)

Testing Function

In [37]:
def test():
    # switches the model to evaluation mode by disabling dropout and bn
    model.eval()
    #these will be used for accuracy
    correct = 0
    total = 0
    #loss accumilation
    total_loss = 0.0
    # Disable gradient computation (saves memory and speeds up inference)
    with torch.no_grad():
        #x is the input and y is the target
        for (x, y) in test_loader:
            #get prediciotns from the model
            outputs = model(x)
            # loss for the batch
            loss = loss_func(outputs, y)  # Compute the loss for the batch
            #loss accumulation
            total_loss += loss.item()
            #this is determining the predicted class
            _, predicted = torch.max(outputs.data, 1)
             # total number of samples
            total += y.size(0)
            # number of correct samples total
            correct += (predicted == y).sum().item()

    #accuracy percentage
    accuracy = 100 * correct / total  # Compute the accuracy percentage
    # return average lostt and accuracy
    return total_loss / len(test_loader), accuracy


Train Accuracy

In [38]:
def train_accuracy():
    #test accuracy and train accuracy are diff so diff functionts
    #but same idea as what's in the test function
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, targets in train_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()

    return 100 * correct / total

In [40]:
def load_model(model_path):
    # loads model for classification
    model.load_state_dict(torch.load(model_path))


Save Model function

In [39]:
def save_model(epoch):
    #saves model in the model folder in model.ckpt
    if not os.path.exists('./model'):
        os.makedirs('./model')
    torch.save(model.state_dict(), './model/model.ckpt')
    print(f"Model saved in file: ./model/model.ckpt")

Load Model Funciont

Train Function


In [41]:
def train():
    #the header
    print(f"{'Loop':<8}{'Train Loss':<15}{'Train Acc %':<15}{'Test Loss':<15}{'Test Acc %':<15}")
    #tracks training acc
    last_train_acc = 0

    # trains for 10 epochs
    for epoch in range(10):
        # sets model to trianing mode
        model.train()
        #counts accumulated loss
        total_loss = 0.0
        #iterates over traing data x is the inputs and y is ground truth
        for x, y in train_loader:
            #clear the gradient from the last batch so no old gradients are used
            optimizer.zero_grad()
            #forward pass
            outputs = model(x)
            #loss computation
            loss = loss_func(outputs, y)
            #back propogation
            loss.backward()
            #this is what updates, it's based on the gradient
            optimizer.step()


            total_loss += loss.item()

        # updates after each epoch
        train_acc = train_accuracy()
        test_loss, test_accuracy = test()

        # makes training accuracy only goes up
        if train_acc < last_train_acc:
         last_train_acc = train_acc

        # prints out results of each epoch
        print(f"{epoch + 1}/10   {total_loss / len(train_loader):<15.4f}{train_acc:<15.2f}{test_loss:<15.4f}{test_accuracy:<15.2f}")

        # Save the model only after the last epoch (10th epoch)
        if epoch == 9:
            save_model(epoch)


Test Function

In [42]:
def classify(image_path):
    # loads the best model from training
    load_model('./model/model.ckpt')
    model.eval()

    # read the image using OpenCV
    img = cv2.imread(image_path)

    if img is None:
        print("Error: Image not found or invalid path.")
        return
    # Convert black/white back to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    #image resizing to make sure it's 32x32
    resized = cv2.resize(img, (32, 32))
    # Convert to tensor and makes it have one channel
    tensor = transform(resized).unsqueeze(0)

    with torch.no_grad():
        #forward pass
        output = model(tensor)
        #prediction of class
        _, predicted_class = torch.max(output.data, 1)
    #which classes it can choose from
    classes = train_data.classes
    #prediction results
    print(f'Prediction result: {classes[predicted_class.item()]}')

Command Line prompts

In [43]:
if __name__ == "__main__":

    command = sys.argv[1]
    # trains the data to create model "python classify.py train"
    if command == "train":
        train()
    # can predict class when given image in class path "python classify.py test ./filepath"
    elif command == "test":
        image_path = sys.argv[2]
        classify(image_path)

Train model

In [47]:
#!python classify.py train

Files already downloaded and verified
Files already downloaded and verified
Loop    Train Loss     Train Acc %    Test Loss      Test Acc %     
1/10   1.9426         37.25          1.7603         37.11          
2/10   1.7045         40.56          1.6785         39.77          
3/10   1.6216         43.56          1.6032         43.13          
4/10   1.5681         44.91          1.5933         43.27          
5/10   1.5234         46.68          1.5427         45.10          
6/10   1.4880         47.59          1.5220         45.52          
7/10   1.4577         48.95          1.5015         46.38          
8/10   1.4310         49.80          1.4929         46.87          
9/10   1.4077         50.25          1.4964         47.45          
10/10   1.3864         51.96          1.4485         48.58          
Model saved in file: ./model/model.ckpt


Test model

In [55]:
#!python classify.py test catfromcifar.png

Files already downloaded and verified
Files already downloaded and verified
/content/classify.py:147: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(tor

MOST RECENT 2/20/2025